In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import torch
import math
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
import torchtuples as tt
from torchtuples import Model
from sklearn.model_selection import train_test_split
from torch import nn, Tensor
import matplotlib.pyplot as plt
from scipy.stats import norm
import os
os.environ["R_HOME"] = f"{os.environ['CONDA_PREFIX']}\\Lib\\R"
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
importr('quantreg')

from pytorchtools import EarlyStopping
import torch.utils.data as Data	# For constructing DataLoader
import torch.nn as nn

from torch import nn, Tensor
from dqAux_new import dqNetSparse,checkLoss
from localrqgrid import lprq0g
from localrq import lprq0
from ph import ph
from bandwidthaic import bandwidthaic
from ci import cifint

In [2]:
#Define functional coefficients
def g10(x): return 1.5*np.exp(-3*(x+1)**2)+1*np.exp(-8*(x-1)**2)
def g11(x): return (0.15*np.exp(4*x)/(1+np.exp(4*x)))
def g12(x): return (0.04*(x)**2)
def g13(x): return (0.15*np.exp(-4*x)/(1+np.exp(-4*x)))
def g14(x): return (0.1*((np.cos(x))**2))
def g15(x): return (0.15*np.exp(4*x)/(1+np.exp(4*x)))
def g16(x): return (0.04*(x)**2)
def g17(x): return (0.15*np.exp(-4*x)/(1+np.exp(-4*x)))
def g18(x): return (0.1*((np.cos(x))**2))

def g20(x): return 1.5*np.exp(-3*(x-1)**2)+1*np.exp(-8*(x+1)**2)
def g21(x): return (0.1*np.sin(-0.8*np.pi*x)+0.1)
def g22(x): return (0.15*np.exp(4*x)/(1+np.exp(4*x)))
def g23(x): return (0.1*np.sin(-0.8*np.pi*x)+0.1)
def g24(x): return (0.15*np.exp(-4*x)/(1+np.exp(-4*x)))
def g25(x): return (0.1*np.sin(-0.8*np.pi*x)+0.1)
def g26(x): return (0.15*np.exp(4*x)/(1+np.exp(4*x)))
def g27(x): return (0.1*np.sin(-0.8*np.pi*x)+0.1)
def g28(x): return (0.15*np.exp(-4*x)/(1+np.exp(-4*x)))

def g30(x): return 1.5*np.exp(-3*(x+1)**2)+1*np.exp(-8*(x-1)**2)
def g31(x): return (0.1*np.sin(0.8*np.pi*x)+0.1)
def g32(x): return (0.1*np.cos(0.8*np.pi*x)+0.1)
def g33(x): return (0.1*np.sin(0.8*np.pi*x)+0.1)
def g34(x): return (0.1*np.cos(0.8*np.pi*x)+0.1)
def g35(x): return (0.1*np.sin(0.8*np.pi*x)+0.1)
def g36(x): return (0.1*np.cos(0.8*np.pi*x)+0.1)
def g37(x): return (0.1*np.sin(0.8*np.pi*x)+0.1)
def g38(x): return (0.1*np.cos(0.8*np.pi*x)+0.1)

def g40(x): return 1.5*np.exp(-3*(x-1)**2)+1*np.exp(-8*(x+1)**2)
def g41(x): return (0.1*np.cos(0.8*np.pi*x)+0.1)
def g42(x): return (0.1*np.sin(0.8*np.pi*x)+0.1)
def g43(x): return (0.1*np.cos(0.8*np.pi*x)+0.1)
def g44(x): return (0.1*np.sin(0.8*np.pi*x)+0.1)
def g45(x): return (0.1*np.cos(0.8*np.pi*x)+0.1)
def g46(x): return (0.1*np.sin(0.8*np.pi*x)+0.1)
def g47(x): return (0.1*np.cos(0.8*np.pi*x)+0.1)
def g48(x): return (0.1*np.sin(0.8*np.pi*x)+0.1)

In [3]:
 tau=0.15
 n=4000
 nt=4000
 nv=800

 #testing data
 index=np.zeros((n,1))
 coefx1=np.zeros((n,9))
 coefx2=np.zeros((n,9))
 coefx3=np.zeros((n,9))
 coefx4=np.zeros((n,9))
  
 Y1=np.zeros((n,1))
 Y1[0]=0
 stdev1=np.zeros((n,1))
 stdev1[0]=1
 Y2=np.zeros((n,1))
 Y2[0]=0
 stdev2=np.zeros((n,1))
 stdev2[0]=1
  
 Y3=np.zeros((n,1))
 Y3[0]=0
 stdev3=np.zeros((n,1))
 stdev3[0]=1
 Y4=np.zeros((n,1))
 Y4[0]=0
 stdev4=np.zeros((n,1))
 stdev4[0]=1

 #training data
 indext=np.zeros((nt,1))
 coefx1t=np.zeros((nt,9))
 coefx2t=np.zeros((nt,9))
 coefx3t=np.zeros((nt,9))
 coefx4t=np.zeros((nt,9))
  
 Y1t=np.zeros((nt,1))
 Y1t[0]=0
 stdev1t=np.zeros((nt,1))
 stdev1t[0]=1
 Y2t=np.zeros((nt,1))
 Y2t[0]=0
 stdev2t=np.zeros((nt,1))
 stdev2t[0]=1
  
 Y3t=np.zeros((nt,1))
 Y3t[0]=0
 stdev3t=np.zeros((nt,1))
 stdev3t[0]=1
 Y4t=np.zeros((nt,1))
 Y4t[0]=0
 stdev4t=np.zeros((nt,1))
 stdev4t[0]=1

 #validation data
 indexv=np.zeros((nv,1))
 coefx1v=np.zeros((nv,9))
 coefx2v=np.zeros((nv,9))
 coefx3v=np.zeros((nv,9))
 coefx4v=np.zeros((nv,9))
  
 Y1v=np.zeros((nv,1))
 Y1v[0]=0
 stdev1v=np.zeros((nv,1))
 stdev1v[0]=1
 Y2v=np.zeros((nv,1))
 Y2v[0]=0
 stdev2v=np.zeros((nv,1))
 stdev2v[0]=1
  
 Y3v=np.zeros((nv,1))
 Y3v[0]=0
 stdev3v=np.zeros((nv,1))
 stdev3v[0]=1
 Y4v=np.zeros((nv,1))
 Y4v[0]=0
 stdev4v=np.zeros((nv,1))
 stdev4v[0]=1

In [4]:
 #testing data
 for j in range(2,(n+1)):
    
     index[j-1]=np.random.uniform(-2,2+10**-10,1)
     e1=np.random.uniform(0,1+10**-10,1)
     e2=np.random.uniform(0,1+10**-10,1)
     e3=np.random.uniform(0,1+10**-10,1)
     e4=np.random.uniform(0,1+10**-10,1)
     e31=(0.2*(e1)**2)+0.8
     e32=(0.2*np.exp(e1))+0.8
     e41=(0.2*(e2)**2)+0.8
     e42=(0.2*np.exp(e2))+0.8
     e51=(0.2*(e3)**2)+0.8
     e52=(0.2*np.exp(e3))+0.8
     e61=(0.2*(e4)**2)+0.8
     e62=(0.2*np.exp(e4))+0.8
    
     e5=np.random.normal(0,1,1)
     e6=np.random.normal(0,1,1)   
     e7=np.random.normal(0,1,1) 
     e8=np.random.normal(0,1,1) 

     coefx1[j-1,]=[g10(index[j-1]).item(),(g11(index[j-1])*e31).item(),(g12(index[j-1])*e32).item(),(g13(index[j-1])*e31).item(),(g14(index[j-1])*e32).item(),g15(index[j-1]).item(),g16(index[j-1]).item(),g17(index[j-1]).item(),g18(index[j-1]).item()]
     coefx2[j-1,]=[g20(index[j-1]).item(),(g21(index[j-1])*e41).item(),(g22(index[j-1])*e42).item(),(g23(index[j-1])*e41).item(),(g24(index[j-1])*e42).item(),g25(index[j-1]).item(),g26(index[j-1]).item(),g27(index[j-1]).item(),g28(index[j-1]).item()]
     coefx3[j-1,]=[g30(index[j-1]).item(),(g31(index[j-1])*e51).item(),(g32(index[j-1])*e52).item(),(g33(index[j-1])*e51).item(),(g34(index[j-1])*e52).item(),g35(index[j-1]).item(),g36(index[j-1]).item(),g37(index[j-1]).item(),g38(index[j-1]).item()]
     coefx4[j-1,]=[g40(index[j-1]).item(),(g41(index[j-1])*e61).item(),(g42(index[j-1])*e62).item(),(g43(index[j-1])*e61).item(),(g44(index[j-1])*e62).item(),g45(index[j-1]).item(),g46(index[j-1]).item(),g47(index[j-1]).item(),g48(index[j-1]).item()]
    
     stdev1[j-1]=np.dot(coefx1[j-1,],[1,stdev1[j-2].item(),stdev2[j-2].item(),stdev3[j-2].item(),stdev4[j-2].item(),np.absolute(Y1[j-2]).item(),np.absolute(Y2[j-2]).item(),np.absolute(Y3[j-2]).item(),np.absolute(Y4[j-2]).item()])
     stdev2[j-1]=np.dot(coefx2[j-1,],[1,stdev1[j-2].item(),stdev2[j-2].item(),stdev3[j-2].item(),stdev4[j-2].item(),np.absolute(Y1[j-2]).item(),np.absolute(Y2[j-2]).item(),np.absolute(Y3[j-2]).item(),np.absolute(Y4[j-2]).item()])
     stdev3[j-1]=np.dot(coefx3[j-1,],[1,stdev1[j-2].item(),stdev2[j-2].item(),stdev3[j-2].item(),stdev4[j-2].item(),np.absolute(Y1[j-2]).item(),np.absolute(Y2[j-2]).item(),np.absolute(Y3[j-2]).item(),np.absolute(Y4[j-2]).item()])
     stdev4[j-1]=np.dot(coefx4[j-1,],[1,stdev1[j-2].item(),stdev2[j-2].item(),stdev3[j-2].item(),stdev4[j-2].item(),np.absolute(Y1[j-2]).item(),np.absolute(Y2[j-2]).item(),np.absolute(Y3[j-2]).item(),np.absolute(Y4[j-2]).item()])
    
     Y1[j-1]=stdev1[j-1]*e5
     Y2[j-1]=stdev2[j-1]*e6
     Y3[j-1]=stdev3[j-1]*e7
     Y4[j-1]=stdev4[j-1]*e8
    

In [5]:
 #training data
 for j in range(2,(nt+1)):
    
     indext[j-1]=np.random.uniform(-2,2+10**-10,1)
     e1t=np.random.uniform(0,1+10**-10,1)
     e2t=np.random.uniform(0,1+10**-10,1)
     e3t=np.random.uniform(0,1+10**-10,1)
     e4t=np.random.uniform(0,1+10**-10,1)
     e31t=(0.2*(e1t)**2)+0.8
     e32t=(0.2*np.exp(e1t))+0.8
     e41t=(0.2*(e2t)**2)+0.8
     e42t=(0.2*np.exp(e2t))+0.8
     e51t=(0.2*(e3t)**2)+0.8
     e52t=(0.2*np.exp(e3t))+0.8
     e61t=(0.2*(e4t)**2)+0.8
     e62t=(0.2*np.exp(e4t))+0.8

     e5t=np.random.normal(0,1,1)
     e6t=np.random.normal(0,1,1)   
     e7t=np.random.normal(0,1,1) 
     e8t=np.random.normal(0,1,1) 

     coefx1t[j-1,]=[g10(indext[j-1]).item(),(g11(indext[j-1])*e31t).item(),(g12(indext[j-1])*e32t).item(),(g13(indext[j-1])*e31t).item(),(g14(indext[j-1])*e32t).item(),g15(indext[j-1]).item(),g16(indext[j-1]).item(),g17(indext[j-1]).item(),g18(indext[j-1]).item()]
     coefx2t[j-1,]=[g20(indext[j-1]).item(),(g21(indext[j-1])*e41t).item(),(g22(indext[j-1])*e42t).item(),(g23(indext[j-1])*e41t).item(),(g24(indext[j-1])*e42t).item(),g25(indext[j-1]).item(),g26(indext[j-1]).item(),g27(indext[j-1]).item(),g28(indext[j-1]).item()]
     coefx3t[j-1,]=[g30(indext[j-1]).item(),(g31(indext[j-1])*e51t).item(),(g32(indext[j-1])*e52t).item(),(g33(indext[j-1])*e51t).item(),(g34(indext[j-1])*e52t).item(),g35(indext[j-1]).item(),g36(indext[j-1]).item(),g37(indext[j-1]).item(),g38(indext[j-1]).item()]
     coefx4t[j-1,]=[g40(indext[j-1]).item(),(g41(indext[j-1])*e61t).item(),(g42(indext[j-1])*e62t).item(),(g43(indext[j-1])*e61t).item(),(g44(indext[j-1])*e62t).item(),g45(indext[j-1]).item(),g46(indext[j-1]).item(),g47(indext[j-1]).item(),g48(indext[j-1]).item()]
    
     stdev1t[j-1]=np.dot(coefx1t[j-1,],[1,stdev1t[j-2].item(),stdev2t[j-2].item(),stdev3t[j-2].item(),stdev4t[j-2].item(),np.absolute(Y1t[j-2]).item(),np.absolute(Y2t[j-2]).item(),np.absolute(Y3t[j-2]).item(),np.absolute(Y4t[j-2]).item()])
     stdev2t[j-1]=np.dot(coefx2t[j-1,],[1,stdev1t[j-2].item(),stdev2t[j-2].item(),stdev3t[j-2].item(),stdev4t[j-2].item(),np.absolute(Y1t[j-2]).item(),np.absolute(Y2t[j-2]).item(),np.absolute(Y3t[j-2]).item(),np.absolute(Y4t[j-2]).item()])
     stdev3t[j-1]=np.dot(coefx3t[j-1,],[1,stdev1t[j-2].item(),stdev2t[j-2].item(),stdev3t[j-2].item(),stdev4t[j-2].item(),np.absolute(Y1t[j-2]).item(),np.absolute(Y2t[j-2]).item(),np.absolute(Y3t[j-2]).item(),np.absolute(Y4t[j-2]).item()])
     stdev4t[j-1]=np.dot(coefx4t[j-1,],[1,stdev1t[j-2].item(),stdev2t[j-2].item(),stdev3t[j-2].item(),stdev4t[j-2].item(),np.absolute(Y1t[j-2]).item(),np.absolute(Y2t[j-2]).item(),np.absolute(Y3t[j-2]).item(),np.absolute(Y4t[j-2]).item()])
    
     Y1t[j-1]=stdev1t[j-1]*e5t
     Y2t[j-1]=stdev2t[j-1]*e6t
     Y3t[j-1]=stdev3t[j-1]*e7t
     Y4t[j-1]=stdev4t[j-1]*e8t
    

In [6]:
 #validation data
 for j in range(2,(nv+1)):
    
     indexv[j-1]=np.random.uniform(-2,2+10**-10,1)
     e1v=np.random.uniform(0,1+10**-10,1)
     e2v=np.random.uniform(0,1+10**-10,1)
     e3v=np.random.uniform(0,1+10**-10,1)
     e4v=np.random.uniform(0,1+10**-10,1)
     e31v=(0.2*(e1v)**2)+0.8
     e32v=(0.2*np.exp(e1v))+0.8
     e41v=(0.2*(e2v)**2)+0.8
     e42v=(0.2*np.exp(e2v))+0.8
     e51v=(0.2*(e3v)**2)+0.8
     e52v=(0.2*np.exp(e3v))+0.8
     e61v=(0.2*(e4v)**2)+0.8
     e62v=(0.2*np.exp(e4v))+0.8
    
     e5v=np.random.normal(0,1,1)
     e6v=np.random.normal(0,1,1)   
     e7v=np.random.normal(0,1,1) 
     e8v=np.random.normal(0,1,1) 

     coefx1v[j-1,]=[g10(indexv[j-1]).item(),(g11(indexv[j-1])*e31v).item(),(g12(indexv[j-1])*e32v).item(),(g13(indexv[j-1])*e31v).item(),(g14(indexv[j-1])*e32v).item(),g15(indexv[j-1]).item(),g16(indexv[j-1]).item(),g17(indexv[j-1]).item(),g18(indexv[j-1]).item()]
     coefx2v[j-1,]=[g20(indexv[j-1]).item(),(g21(indexv[j-1])*e41v).item(),(g22(indexv[j-1])*e42v).item(),(g23(indexv[j-1])*e41v).item(),(g24(indexv[j-1])*e42v).item(),g25(indexv[j-1]).item(),g26(indexv[j-1]).item(),g27(indexv[j-1]).item(),g28(indexv[j-1]).item()]
     coefx3v[j-1,]=[g30(indexv[j-1]).item(),(g31(indexv[j-1])*e51v).item(),(g32(indexv[j-1])*e52v).item(),(g33(indexv[j-1])*e51v).item(),(g34(indexv[j-1])*e52v).item(),g35(indexv[j-1]).item(),g36(indexv[j-1]).item(),g37(indexv[j-1]).item(),g38(indexv[j-1]).item()]
     coefx4v[j-1,]=[g40(indexv[j-1]).item(),(g41(indexv[j-1])*e61v).item(),(g42(indexv[j-1])*e62v).item(),(g43(indexv[j-1])*e61v).item(),(g44(indexv[j-1])*e62v).item(),g45(indexv[j-1]).item(),g46(indexv[j-1]).item(),g47(indexv[j-1]).item(),g48(indexv[j-1]).item()]
    
     stdev1v[j-1]=np.dot(coefx1v[j-1,],[1,stdev1v[j-2].item(),stdev2v[j-2].item(),stdev3v[j-2].item(),stdev4v[j-2].item(),np.absolute(Y1v[j-2]).item(),np.absolute(Y2v[j-2]).item(),np.absolute(Y3v[j-2]).item(),np.absolute(Y4v[j-2]).item()])
     stdev2v[j-1]=np.dot(coefx2v[j-1,],[1,stdev1v[j-2].item(),stdev2v[j-2].item(),stdev3v[j-2].item(),stdev4v[j-2].item(),np.absolute(Y1v[j-2]).item(),np.absolute(Y2v[j-2]).item(),np.absolute(Y3v[j-2]).item(),np.absolute(Y4v[j-2]).item()])
     stdev3v[j-1]=np.dot(coefx3v[j-1,],[1,stdev1v[j-2].item(),stdev2v[j-2].item(),stdev3v[j-2].item(),stdev4v[j-2].item(),np.absolute(Y1v[j-2]).item(),np.absolute(Y2v[j-2]).item(),np.absolute(Y3v[j-2]).item(),np.absolute(Y4v[j-2]).item()])
     stdev4v[j-1]=np.dot(coefx4v[j-1,],[1,stdev1v[j-2].item(),stdev2v[j-2].item(),stdev3v[j-2].item(),stdev4v[j-2].item(),np.absolute(Y1v[j-2]).item(),np.absolute(Y2v[j-2]).item(),np.absolute(Y3v[j-2]).item(),np.absolute(Y4v[j-2]).item()])
    
     Y1v[j-1]=stdev1v[j-1]*e5v
     Y2v[j-1]=stdev2v[j-1]*e6v
     Y3v[j-1]=stdev3v[j-1]*e7v
     Y4v[j-1]=stdev4v[j-1]*e8v
    

In [7]:
 #testing data
 Y1=Y1[1:n]
 Y2=Y2[1:n]
 Y3=Y3[1:n]
 Y4=Y4[1:n]
 stdev1=stdev1[1:n]
 stdev2=stdev2[1:n]
 stdev3=stdev3[1:n]
 stdev4=stdev4[1:n]
 index=index[1:n]
 n=np.size(index,0)

 #training data
 Y1t=Y1t[1:nt]
 Y2t=Y2t[1:nt]
 Y3t=Y3t[1:nt]
 Y4t=Y4t[1:nt]
 stdev1t=stdev1t[1:nt]
 stdev2t=stdev2t[1:nt]
 stdev3t=stdev3t[1:nt]
 stdev4t=stdev4t[1:nt]
 indext=indext[1:nt]
 nt=np.size(indext,0)

 #validation data
 Y1v=Y1v[1:nv]
 Y2v=Y2v[1:nv]
 Y3v=Y3v[1:nv]
 Y4v=Y4v[1:nv]
 stdev1v=stdev1v[1:nv]
 stdev2v=stdev2v[1:nv]
 stdev3v=stdev3v[1:nv]
 stdev4v=stdev4v[1:nv]
 indexv=indexv[1:nv]
 nv=np.size(indexv,0)

 ymax1=np.max(Y1)
 ymax2=np.max(Y2)
 ymin1=np.min(Y1)
 ymin2=np.min(Y2)
 ymax3=np.max(Y3)
 ymax4=np.max(Y4)
 ymin3=np.min(Y3)
 ymin4=np.min(Y4)

 yrange1=np.max(ymax1)-np.min(ymin1)
 yrange2=np.max(ymax2)-np.min(ymin2)
 yrange3=np.max(ymax3)-np.min(ymin3)
 yrange4=np.max(ymax4)-np.min(ymin4)

In [15]:
 #Preliminary step
 L=int(np.floor(0.8*(n^(1/8)))
 preY=np.hstack((Y1,Y2,Y3,Y4))
 Y=np.hstack((Y1[(L+1):(n-1)],Y2[(L+1):(n-1)],Y3[(L+1):(n-1)],Y4[(L+1):(n-1)]))
 indexz=index[(L+1):(n-1)]
 kap=np.size(preY,1)
 preY11=np.zeros(((n-L-2),(kap*(L+1))))
 indexzz=np.zeros(((n-L-2),(L+1)))

 preYt=np.hstack((Y1t,Y2t,Y3t,Y4t))
 Yt=np.hstack((Y1t[(L+1):(nt-1)],Y2t[(L+1):(nt-1)],Y3t[(L+1):(nt-1)],Y4t[(L+1):(nt-1)]))
 indexzt=index[(L+1):(nt-1)]
 kap=np.size(preYt,1)
 preY11t=np.zeros(((nt-L-2),(kap*(L+1))))
 indexzzt=np.zeros(((nt-L-2),(L+1)))

 preYv=np.hstack((Y1v,Y2v,Y3v,Y4v))
 Yv=np.hstack((Y1v[(L+1):(nv-1)],Y2v[(L+1):(nv-1)],Y3v[(L+1):(nv-1)],Y4v[(L+1):(nv-1)]))
 indexzv=indexv[(L+1):(nv-1)]
 kap=np.size(preYv,1)
 preY11v=np.zeros(((nv-L-2),(kap*(L+1))))
 indexzzv=np.zeros(((nv-L-2),(L+1)))
  
 for lg in range(L+1):
    
   preY11[:,(kap*lg):(kap*(lg+1))]=np.absolute(preY[((L+1)-lg-1):(n-1-lg-1),:])
   indexzz[:,lg]=index[((L+1)-(lg)):((n-1)-(lg))].reshape(-1)

   preY11t[:,(kap*lg):(kap*(lg+1))]=np.absolute(preYt[((L+1)-lg-1):(nt-1-lg-1),:])
   indexzzt[:,lg]=indext[((L+1)-(lg)):((nt-1)-(lg))].reshape(-1)

   preY11v[:,(kap*lg):(kap*(lg+1))]=np.absolute(preYv[((L+1)-lg-1):(nv-1-lg-1),:])
   indexzzv[:,lg]=indexv[((L+1)-(lg)):((nv-1)-(lg))].reshape(-1)


In [16]:
 nm0=np.zeros((L+2,1))
 for l in range(L+1):
   nm0[(l+1)]=nm0[l]+(l+1)+kap
 nm=nm0.astype(int)

In [17]:
 x_trainin=np.zeros((nt-L-2,nm[L+1][0]))
 for ll in range(L+1):
   x_trainin[:,(nm[ll][0]):nm[(ll+1)][0]]=np.hstack((indexzzt[:,0:(ll+1)],preY11t[:,(kap*ll):(kap*(ll+1))]))

 x_testin=np.zeros((n-L-2,nm[L+1][0]))
 for ll in range(L+1):
   x_testin[:,(nm[ll][0]):nm[(ll+1)][0]]=np.hstack((indexzz[:,0:(ll+1)],preY11[:,(kap*ll):(kap*(ll+1))]))

 x_valin=np.zeros((nv-L-2,nm[L+1][0]))
 for ll in range(L+1):
   x_valin[:,(nm[ll][0]):nm[(ll+1)][0]]=np.hstack((indexzzv[:,0:(ll+1)],preY11v[:,(kap*ll):(kap*(ll+1))]))

In [18]:
 #Convert data into tensor for DNN
 ##testing data
 Y_1=torch.tensor(Y[:,0].reshape(-1,1).astype('float32'))
 Y_2=torch.tensor(Y[:,1].reshape(-1,1).astype('float32'))
 Y_3=torch.tensor(Y[:,2].reshape(-1,1).astype('float32'))
 Y_4=torch.tensor(Y[:,3].reshape(-1,1).astype('float32'))
 indexz=torch.tensor(indexz.astype('float32'))

 preY11=torch.tensor(preY11.astype('float32'))
 indexzz=torch.tensor(indexzz.astype('float32'))
 #x_test=torch.cat((indexzz,preY11),1) 
 x_test=torch.tensor(x_testin.astype('float32'))

 ##training data
 Y_1t=torch.tensor(Yt[:,0].reshape(-1,1).astype('float32'))
 Y_2t=torch.tensor(Yt[:,1].reshape(-1,1).astype('float32'))
 Y_3t=torch.tensor(Yt[:,2].reshape(-1,1).astype('float32'))
 Y_4t=torch.tensor(Yt[:,3].reshape(-1,1).astype('float32'))
 indexzt=torch.tensor(indexzt.astype('float32'))

 preY11t=torch.tensor(preY11t.astype('float32'))
 indexzzt=torch.tensor(indexzzt.astype('float32'))
 #x_train=torch.cat((indexzzt,preY11t),1) 
 x_train=torch.tensor(x_trainin.astype('float32'))

 ##validation data
 Y_1v=torch.tensor(Yv[:,0].reshape(-1,1).astype('float32'))
 Y_2v=torch.tensor(Yv[:,1].reshape(-1,1).astype('float32'))
 Y_3v=torch.tensor(Yv[:,2].reshape(-1,1).astype('float32'))
 Y_4v=torch.tensor(Yv[:,3].reshape(-1,1).astype('float32'))
 indexzv=torch.tensor(indexzv.astype('float32'))

 preY11v=torch.tensor(preY11v.astype('float32'))
 indexzzv=torch.tensor(indexzzv.astype('float32'))
 x_val=torch.tensor(x_valin.astype('float32'))
 #x_val=torch.cat((indexzzv,preY11v),1) 

In [19]:
 # training Quantile Regression
 #dim_nonpar=np.size(x_trainin,1)
 dim_nonpar=np.size(x_train,1)

 # hyperparameters
 nodes = [2,128]     # number of network layers and nodes per layer
 lr = 0.001          #learning rate
 epochs = 500
 verbose = False
 sparseRatio = 0.5   #proportion of 0 elements in weight matrix

 ## loss function
 def check_loss(y_pred: Tensor, target: Tensor, tau: float) -> Tensor:
    errors = target-y_pred 
    u = torch.max(tau*errors,(tau-1)*errors)
    return u.mean()

 EPOCH = epochs

 reslist1 = []
 Train_Loss_list1 = []
 Valid_Loss_list1 = []

 Net1=dqNetSparse(dim_nonpar,nodes,sparseRatio)
 optimizer1=torch.optim.Adam(Net1.parameters(),lr=lr)

 #training step
 model_path="E:/learning_python/weights"   #path of saving model file in each epoch

 for epoch in range(1, EPOCH + 1):

    prediction1t=Net1(x_train) 
    train_loss1 = check_loss(prediction1t,Y_1t,tau) 
    optimizer1.zero_grad() 
    train_loss1.backward() 
    optimizer1.step() 
     
    Train_Loss_list1.append(train_loss1)
    torch.save(Net1, os.path.join(model_path, 'catdogclassifi_0.15_4000_1-%s.pth' % epoch))  #saving model file in each epoch
     
    # validation
    y_fval1=Net1(x_val)
    test_loss1= check_loss(y_fval1,Y_1v,tau)
    Valid_Loss_list1.append(test_loss1)
    reslist1.append(test_loss1)
    print(epoch)    

 min_num1 = min(reslist1)
 min_index1 = reslist1.index(min_num1)
 #min_num1 = min(Train_Loss_list1)
 #min_index1 = Train_Loss_list1.index(min_num1)
 print('model%s' % (min_index1 + 1))              #print the number of order of the best model

 Netnew1=torch.load(r'E:/learning_python/weights/catdogclassifi_0.15_4000_1-%s.pth' % (min_index1 + 1))  #save the best model
 sigmatile1=Netnew1(x_test).detach().numpy()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


C:\Users\Administrator\AppData\Local\Temp\ipykernel_6536\1503340234.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Netnew1=torch.load(r'E:/learning_python/weights/catd

In [20]:
 ##Y_2

 reslist2 = []
 Train_Loss_list2 = []
 Valid_Loss_list2 = []

 Net2=dqNetSparse(dim_nonpar,nodes,sparseRatio)
 optimizer2=torch.optim.Adam(Net2.parameters(),lr=lr)

 #training step
 model_path="E:/learning_python/weights"
 
 for epoch in range(1, EPOCH + 1):

    prediction2t=Net2(x_train) 
    train_loss2 = check_loss(prediction2t,Y_2t,tau) 
    optimizer2.zero_grad() 
    train_loss2.backward() 
    optimizer2.step() 
     
    Train_Loss_list2.append(train_loss2)
    torch.save(Net2, os.path.join(model_path, 'catdogclassifi_0.15_4000_2-%s.pth' % epoch))
     
    y_fval2=Net2(x_val)
    test_loss2= check_loss(y_fval2,Y_2v,tau)
    Valid_Loss_list2.append(test_loss2)
    reslist2.append(test_loss2)
    print(epoch)
     
 min_num2 = min(reslist2)
 min_index2 = reslist2.index(min_num2)
 #min_num2 = min(Train_Loss_list2)
 #min_index2 = Train_Loss_list2.index(min_num2)
 print('model%s' % (min_index2 + 1))

 Netnew2=torch.load(r'E:/learning_python/weights/catdogclassifi_0.15_4000_2-%s.pth' % (min_index2 + 1))
 sigmatile2=Netnew2(x_test).detach().numpy()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


C:\Users\Administrator\AppData\Local\Temp\ipykernel_6536\3915685258.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Netnew2=torch.load(r'E:/learning_python/weights/catd

In [21]:
 ##Y_3
 reslist3 = []
 Train_Loss_list3 = []
 Valid_Loss_list3 = []

 Net3=dqNetSparse(dim_nonpar,nodes,sparseRatio)
 optimizer3=torch.optim.Adam(Net3.parameters(),lr=lr)

 # training step
 model_path="E:/learning_python/weights"

 for epoch in range(1, EPOCH + 1):

    prediction3t=Net3(x_train) 
    train_loss3 = check_loss(prediction3t,Y_3t,tau) 
    optimizer3.zero_grad() 
    train_loss3.backward() 
    optimizer3.step() 

    Train_Loss_list3.append(train_loss3)
    torch.save(Net3, os.path.join(model_path, 'catdogclassifi_0.15_4000_3-%s.pth' % epoch))
     
    y_fval3=Net3(x_val)
    test_loss3= check_loss(y_fval3,Y_3v,tau)
    Valid_Loss_list3.append(test_loss3)
    reslist3.append(test_loss3)
     
    print(epoch)
     
 min_num3 = min(reslist3)
 min_index3 = reslist3.index(min_num3)
 #min_num3 = min(Train_Loss_list3)
 #min_index3 = Train_Loss_list3.index(min_num3)
 print('model%s' % (min_index3 + 1))

 Netnew3=torch.load(r'E:/learning_python/weights/catdogclassifi_0.15_4000_3-%s.pth' % (min_index3 + 1))
 sigmatile3=Netnew3(x_test).detach().numpy()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


C:\Users\Administrator\AppData\Local\Temp\ipykernel_6536\3204006099.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Netnew3=torch.load(r'E:/learning_python/weights/catd

In [22]:
 ##Y_4
 reslist4 = []
 Train_Loss_list4 = []
 Valid_Loss_list4 = []

 Net4=dqNetSparse(dim_nonpar,nodes,sparseRatio)
 optimizer4=torch.optim.Adam(Net4.parameters(),lr=lr)

 # training step
 model_path="E:/learning_python/weights"

 for epoch in range(1, EPOCH + 1):

    prediction4t=Net4(x_train) 
    train_loss4 = check_loss(prediction4t,Y_4t,tau) 
    optimizer4.zero_grad() 
    train_loss4.backward() 
    optimizer4.step()     

    Train_Loss_list4.append(train_loss4)
    torch.save(Net4, os.path.join(model_path, 'catdogclassifi_0.15_4000_4-%s.pth' % epoch))
     
    y_fval4=Net4(x_val)
    test_loss4= check_loss(y_fval4,Y_4v,tau)
    Valid_Loss_list4.append(test_loss4)
    reslist4.append(test_loss4)
    print(epoch)
     
 min_num4 = min(reslist4)
 min_index4 = reslist4.index(min_num4)
 #min_num4 = min(Train_Loss_list4)
 #min_index4 = Train_Loss_list4.index(min_num4)
 print('model%s' % (min_index4 + 1))

 Netnew4=torch.load(r'E:/learning_python/weights/catdogclassifi_0.15_4000_4-%s.pth' % (min_index4 + 1))
 sigmatile4=Netnew4(x_test).detach().numpy()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


C:\Users\Administrator\AppData\Local\Temp\ipykernel_6536\2284064140.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Netnew4=torch.load(r'E:/learning_python/weights/catd

In [23]:
min_index1 + 1

239

In [24]:
min_index2 + 1

284

In [25]:
min_index3 + 1

139

In [26]:
min_index4 + 1

187